In [1]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 100

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, offline
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

import requests

from os import listdir
import unicodedata

from paths import *
from scripts.manipulation import remove_acentos
from scripts.manipulation import normalize_cols
from scripts import scrap_data
from scripts import manipulation
from scripts import io
from scripts import vis_graphs

from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

import yaml


In [12]:
sp_full = pd.read_csv('https://raw.githubusercontent.com/seade-R/dados-covid-sp/master/data/dados_covid_sp.csv', sep=';')
sp_full['nome_drs'] = np.where(sp_full['nome_munic']=='São Paulo','Município de São Paulo',sp_full['nome_drs'])

cols = ['datahora','nome_munic','casos','obitos','nome_drs']
sp = sp_full[cols][sp_full['nome_drs'].notnull()]

sp = sp.groupby(by=['nome_drs','datahora'], as_index=False).sum().sort_values(by=['nome_drs','datahora'])

estado = sp.groupby(by=['datahora'], as_index=False).sum().sort_values(by=['datahora'])
estado['nome_drs'] = 'Estado de São Paulo'
estado = estado[sp.columns]

sp = pd.concat([sp,estado],axis=0)

In [13]:
sp['casos'] = sp['casos'].astype(int)
sp['obitos'] = sp['obitos'].astype(int)

In [14]:
sp['nome_drs'].unique()

array(['Araraquara', 'Araçatuba', 'Baixada Santista', 'Barretos', 'Bauru',
       'Campinas', 'Franca', 'Grande São Paulo', 'Marília',
       'Município de São Paulo', 'Piracicaba', 'Presidente Prudente',
       'Registro', 'Ribeirão Preto', 'Sorocaba', 'São José do Rio Preto',
       'São João da Boa Vista', 'Taubaté', 'Estado de São Paulo'],
      dtype=object)

In [15]:
dd_final_all = pd.DataFrame()

for drs in sp['nome_drs'].unique():

    tbt = sp[sp['nome_drs']==drs]

    tbt['casos_shift'] = tbt['casos'].shift(7)
    tbt['obitos_shift'] = tbt['obitos'].shift(7)

    tbt['casos_7'] = tbt['casos'] - tbt['casos_shift']
    tbt['obitos_7'] = tbt['obitos'] - tbt['obitos_shift']

    tbt['casos_7_shift'] = tbt['casos_7'].shift(7)
    tbt['obitos_7_shift'] = tbt['obitos_7'].shift(7)

    tbt['casos_var'] = tbt['casos_7']/tbt['casos_7_shift']
    tbt['obitos_var'] = tbt['obitos_7']/tbt['obitos_7_shift']

    dd = tbt[['datahora','nome_drs','casos_var','obitos_var']]

    dd_final_all = pd.concat([dd_final_all,dd], axis=0)

    
mask = dd_final_all['datahora'] == max(dd_final_all['datahora'])
dd_final = dd_final_all[mask]

In [16]:
sp_cv = pd.read_csv('https://github.com/seade-R/dados-covid-sp/raw/master/data/plano_sp_leitos_internacoes.csv', sep=';')
sp_cv['internacoes_var'] = sp_cv['internacoes_7d']/sp_cv['internacoes_7d_l']
cols = ['datahora','nome_drs','leitos_pc','internacoes_var']
uti_all = sp_cv[cols]

mask = uti_all['datahora'] == max(uti_all['datahora'])
df_final = uti_all[mask][cols]
df_final['nome_drs'] = df_final['nome_drs'].apply(lambda x: x[7:] if x[:3]=='DRS' else x).str.strip()


In [17]:
uti = pd.read_csv('../data/sp_gov/uti_ocupacao.csv', sep=';')
uti_final = uti[uti['datahora']==max(uti['datahora'])]

In [38]:

df = pd.merge(dd_final,df_final,on=['datahora','nome_drs'],how='outer').merge(uti_final,on=['datahora','nome_drs'],how='outer')
mask = df['nome_drs'].str.startswith('Grande')
df = df[np.logical_not(mask)]
df['leitos_pc'] = df['leitos_pc'].str.replace(',','.')

df = df.round(2)

def uti_fase(x):
    if x>=80:
        fase=1
    elif (x <80) & (x>=70):
        fase=2
    elif (x <70) & (x>=60):
        fase=3
    else:
        fase = 4
    return fase

def leitos_fase(x):
    if x<=3:
        fase=1
    elif (x >3) & (x<=5):
        fase=2
    else:
        fase = 4
    return fase


O = df['uti_var'].apply(lambda x: uti_fase(x))

L = df['leitos_pc'].str.replace(',','.').astype(float).apply(lambda x: leitos_fase(x))


c1 = (O*4 + L)/5

df['Capacidade Hospitalar'] = c1.astype(int)

def casos_fase(x):
    if x>=2:
        fase=1
    elif (x <2) & (x>=1):
        fase=3
    else:
        fase = 4
    return fase

def internacoes_fase(x):
    if x>=1.5:
        fase=1
    elif (x <1.5) & (x>=1):
        fase=2
    elif (x <1) & (x>=0.5):
        fase=3
    else:
        fase = 4
    return fase

def obitos_fase(x):
    if x>=2:
        fase=1
    elif (x <2) & (x>=1):
        fase=2
    elif (x <1) & (x>=0.5):
        fase=3
    else:
        fase = 4
    return fase
NC = df['casos_var'].apply(lambda x: casos_fase(x))

NI = df['internacoes_var'].apply(lambda x: internacoes_fase(x))

NO = df['obitos_var'].apply(lambda x: obitos_fase(x))

c2 = (NC + 3*NI + NO)/5

df['Evolução da Pandemia'] = c2

df['casos_var'] = df['casos_var'].round(2).astype(str)
df['obitos_var'] = df['obitos_var'].round(2).astype(str)
df['internacoes_var'] = df['internacoes_var'].round(2).astype(str)
df['uti_var'] = df['uti_var'].round(2).astype(str)

df['Classif. Final'] = df[['Capacidade Hospitalar','Evolução da Pandemia']].min(axis=1)

df['Classif. Final'] = df['Classif. Final'].astype(int).astype(str)
df['Evolução da Pandemia'] = df['Evolução da Pandemia'].astype(int).astype(str)
df['Capacidade Hospitalar'] = df['Capacidade Hospitalar'].astype(int).astype(str)

In [39]:

cols = ['datahora','nome_drs','Capacidade Hospitalar','uti_var','leitos_pc','Evolução da Pandemia','casos_var','internacoes_var','obitos_var','Classif. Final']


rename_drs = {
    'Araraquara':'DRS 03 - Araraquara',
    'Araçatuba':'DRS 02 - Araçatuba',
    'Baixada Santista':'DRS 04 - Baixada Santista',
    'Barretos':'DRS 05 - Barretos',
    'Bauru':'DRS 06 - Bauru',
    'Campinas':'DRS 07 - Campinas',
    'Estado de São Paulo':'Estado de São Paulo',
    'Franca':'DRS 08 - Franca',
    'Marília':'DRS 09 - Marília',
    'Município de São Paulo':'DRS 01 - Município de São Paulo',
    'Piracicaba':'DRS 10 - Piracicaba',
    'Presidente Prudente':'DRS 11 - Presidente Prudente',
    'Registro':'DRS 12 - Registro',
    'Ribeirão Preto':'DRS 13 - Ribeirão Preto',
    'Sorocaba':'DRS 16 - Sorocaba',
    'São José do Rio Preto':'DRS 15 - São José do Rio Preto',
    'São João da Boa Vista':'DRS 14 - São João da Boa Vista',
    'Taubaté':'DRS 17 - Taubaté'
}

df['nome_drs'] = df['nome_drs'].map(rename_drs)

cols_rename ={
    'datahora':'Data',
    'nome_drs':'DRS',
    'casos_var':'Variação de casos',
    'obitos_var':'Variação de óbitos',
    'leitos_pc':'Leitos COVID/100 mil hab',
    'internacoes_var':'Variação internações',
    'uti_var':'Ocupação leitos UTI COVID',
}




df = df[cols].rename(columns=cols_rename).sort_values(by='DRS')


In [40]:
df

,Data,DRS,Capacidade Hospitalar,Ocupação leitos UTI COVID,Leitos COVID/100 mil hab,Evolução da Pandemia,Variação de casos,Variação internações,Variação de óbitos,Classif. Final
9,2020-07-09,DRS 01 - Município de São Paulo,3,66.6,36.17,3,0.67,0.97,1.03,3
1,2020-07-09,DRS 02 - Araçatuba,3,60.4,12.60,1,1.54,1.61,1.47,1
0,2020-07-09,DRS 03 - Araraquara,4,31.2,9.74,2,0.98,1.45,1.5,2
2,2020-07-09,DRS 04 - Baixada Santista,4,49.4,26.17,2,1.29,0.98,1.04,2
3,2020-07-09,DRS 05 - Barretos,2,78.0,14.38,2,0.91,1.13,0.72,2
4,2020-07-09,DRS 06 - Bauru,3,65.6,10.23,2,0.81,1.15,0.95,2
5,2020-07-09,DRS 07 - Campinas,1,80.4,17.99,2,1.0,0.98,1.31,1
6,2020-07-09,DRS 08 - Franca,1,84.7,7.12,2,1.33,1.0,1.57,1
8,2020-07-09,DRS 09 - Marília,4,10.9,10.89,2,1.43,0.92,1.36,2
10,2020-07-09,DRS 10 - Piracicaba,2,78.4,13.36,2,1.31,1.15,2.0,2


In [41]:
def cases_color(val):
    if val == '1':
        color = 'red'
    elif val=='2':
        color = 'orange'
    elif val=='3':
        color = 'yellow'
    else:
        color = 'green'
    return 'background-color: %s' % color

df.style.applymap(cases_color, subset=pd.IndexSlice[:, ['Capacidade Hospitalar','Evolução da Pandemia','Classif. Final']] )

,Data,DRS,Capacidade Hospitalar,Ocupação leitos UTI COVID,Leitos COVID/100 mil hab,Evolução da Pandemia,Variação de casos,Variação internações,Variação de óbitos,Classif. Final
9,2020-07-09,DRS 01 - Município de São Paulo,3,66.6,36.17,3,0.67,0.97,1.03,3
1,2020-07-09,DRS 02 - Araçatuba,3,60.4,12.60,1,1.54,1.61,1.47,1
0,2020-07-09,DRS 03 - Araraquara,4,31.2,9.74,2,0.98,1.45,1.5,2
2,2020-07-09,DRS 04 - Baixada Santista,4,49.4,26.17,2,1.29,0.98,1.04,2
3,2020-07-09,DRS 05 - Barretos,2,78.0,14.38,2,0.91,1.13,0.72,2
4,2020-07-09,DRS 06 - Bauru,3,65.6,10.23,2,0.81,1.15,0.95,2
5,2020-07-09,DRS 07 - Campinas,1,80.4,17.99,2,1.0,0.98,1.31,1
6,2020-07-09,DRS 08 - Franca,1,84.7,7.12,2,1.33,1.0,1.57,1
8,2020-07-09,DRS 09 - Marília,4,10.9,10.89,2,1.43,0.92,1.36,2
10,2020-07-09,DRS 10 - Piracicaba,2,78.4,13.36,2,1.31,1.15,2.0,2


In [43]:
mask = dd_final_all['nome_drs']=='Baixada Santista'
dd_final_all[mask].tail(30)

,datahora,nome_drs,casos_var,obitos_var
378,2020-06-10,Baixada Santista,1.071383,0.771930
379,2020-06-11,Baixada Santista,1.002977,0.655172
380,2020-06-12,Baixada Santista,0.884809,0.808081
381,2020-06-13,Baixada Santista,0.739733,0.594340
382,2020-06-14,Baixada Santista,0.863311,0.663462
383,2020-06-15,Baixada Santista,0.984536,0.728972
384,2020-06-16,Baixada Santista,1.101481,0.845361
385,2020-06-17,Baixada Santista,0.897680,1.159091
386,2020-06-18,Baixada Santista,0.745622,1.421053
387,2020-06-19,Baixada Santista,1.151093,1.875000
